In [1]:
from pynq import Overlay
import numpy as np
import time

ol = Overlay("./overlay/adder.bit")

In [2]:
ol?

In [3]:
print(ol.ip_dict.keys())

dict_keys(['Adder', 'processing_system7_0'])


In [4]:
adder = ol.Adder

In [5]:
adder.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  A = Register(A=write-only),
  B = Register(B=write-only),
  C = Register(C=write-only),
  Y = Register(Y=0),
  Y_ctrl = Register(Y_ap_vld=0, RESERVED=0)
}

In [9]:
a = 1979
b = 112
c = 9303

In [11]:
adder.write(0x10, a)
adder.write(0x18, b)
adder.write(0x20, c)

print("Running FPGA adder...")
adder.write(0x00, 1)

t1 = time.perf_counter()
while (adder.read(0x00) & 0x2) == 0:
    pass
t2 = time.perf_counter()

print(f"FPGA Adder run time: {(t2 - t1)*1e3:.3f} ms")

Running FPGA adder...
FPGA Adder run time: 1.051 ms


In [12]:
adder.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  A = Register(A=write-only),
  B = Register(B=write-only),
  C = Register(C=write-only),
  Y = Register(Y=4294960196),
  Y_ctrl = Register(Y_ap_vld=1, RESERVED=0)
}

In [13]:
print(a + 2 * b - c)

-7100


In [16]:
val = adder.read(0x28)
val

4294960196

In [18]:
def uint_to_int(num, bits):
    if (num & (1 << (bits - 1))) != 0:
        num = num - (1 << bits)
    return num

In [19]:
print(uint_to_int(val, 32))

-7100


In [20]:
import random

for i in range(10):
    a = random.randint(1, 999)
    b = random.randint(1, 999)
    c = random.randint(1, 9999)
    adder.write(0x10, a)
    adder.write(0x18, b)
    adder.write(0x20, c)
    
    adder.write(0x00, 1)
    while (adder.read(0x00) & 0x2) == 0:
        pass
    
    ground_truth = a + 2 * b - c
    val = adder.read(0x28)
    
    val = uint_to_int(val, 32)
    
    if (val != ground_truth):
        print(f"Error!")
        print(f"val: {val} != ground_truth: {ground_truth}")